<a href="https://colab.research.google.com/github/JordanDCunha/R-for-Data-Science-2e-/blob/main/Chapter_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧹 **5.1 Introduction — Tidy Data**

> **“Happy families are all alike; every unhappy family is unhappy in its own way.”**  
> — *Leo Tolstoy*  
>
> **“Tidy datasets are all alike, but every messy dataset is messy in its own way.”**  
> — *Hadley Wickham*

---

In this chapter, you’ll learn a **consistent and powerful way to organize data in R** using a system called **✨ tidy data ✨**.

Getting your data into a tidy format often requires some **up-front effort**, but that effort pays off quickly. Once your data is tidy—and paired with the **tidy tools** from the **tidyverse**—you’ll spend **far less time munging data** and **far more time answering meaningful questions**.

---

## 🎯 **Chapter Goals**

In this chapter, you will:

- 📐 Learn the **definition of tidy data**
- 🧸 See tidy data applied to a **simple toy dataset**
- 🔄 Master **pivoting**, the primary tool for tidying data  
  - Pivoting changes the **shape** of your data  
  - Pivoting does **not** change the **values**

---

## 📦 **5.1.1 Prerequisites**

This chapter focuses on **`tidyr`**, a package designed to help you **tidy messy datasets**.  
`tidyr` is part of the **core tidyverse**, so loading the tidyverse gives you everything you need.

From this chapter onward, we will **suppress the tidyverse startup messages** to keep output clean and readable.


In [ ]:
library(tidyverse)


# 🧼 **5.2 Tidy Data**

You can represent the **same underlying data** in multiple ways. The examples below all describe the same information using four variables:

- **country**
- **year**
- **population**
- **number of documented TB cases**

However, each dataset organizes those values **differently**.

---

## 📊 **Three Representations of the Same Data**

### **`table1`**
- Each **variable is a column**
- Each **observation is a row**
- This format is **tidy**

### **`table2`**
- Variables are split across **rows**
- Requires reshaping before analysis

### **`table3`**
- Multiple values are stored in a **single cell**
- Not directly usable for computation

Although all three contain the same data, **`table1` is the easiest to work with** because it follows the principles of **tidy data**.

---

## ✅ **The Three Rules of Tidy Data**

A dataset is **tidy** if it satisfies all three rules:

1. **Each variable is a column**
2. **Each observation is a row**
3. **Each value is a cell**

These rules are interconnected and reinforce one another.

---

## 🎯 **Why Tidy Data Matters**

There are **two major advantages** to tidying your data:

### 🔁 **Consistency**
Using one consistent data structure makes it easier to:
- Learn tools
- Read code
- Reuse workflows

### ⚡ **Vectorization**
Placing variables in columns allows R’s **vectorized operations** to shine.  
Most R functions work naturally on vectors, making transformations clearer and faster.

---

## 🧰 **Tidyverse Compatibility**

Packages like **dplyr** and **ggplot2** are designed to work with **tidy data**, which makes common tasks like transforming, summarizing, and visualizing data straightforward.

Below are a few examples using **`table1`**.

---

## ✍️ **5.2.1 Exercises**

1. For **each table**, describe:
   - What an **observation** represents
   - What each **column** represents

2. Sketch the steps needed to compute TB rates for **`table2`** and **`table3`**:
   - Extract TB cases per country per year
   - Extract population per country per year
   - Compute `cases / population * 10000`
   - Store the result appropriately

You don’t yet know all the functions required—but you *do* know how to reason through the transformations.


In [ ]:
# Compute TB rate per 10,000 people
table1 |>
  mutate(rate = cases / population * 10000)

# Compute total TB cases per year
table1 |>
  group_by(year) |>
  summarize(total_cases = sum(cases))

# Visualize changes in TB cases over time
ggplot(table1, aes(x = year, y = cases)) +
  geom_line(aes(group = country), color = "grey50") +
  geom_point(aes(color = country, shape = country)) +
  scale_x_continuous(breaks = c(1999, 2000))


# 🔄 **5.3 Lengthening Data**

Most real-world datasets are **untidy**. Two common reasons explain why:

- Data is often organized to support **data entry**, not analysis.
- Many people are unfamiliar with **tidy data principles**.

As a result, almost every real analysis requires **tidying**. The general workflow is:

1. Identify the **underlying variables** and **observations**
2. **Pivot** the data so that variables live in columns and observations live in rows

The **tidyr** package provides two essential pivoting tools:
- **`pivot_longer()`** → makes data *longer*
- **`pivot_wider()`** → makes data *wider*

We begin with **`pivot_longer()`**, the most common operation.

---

## 📊 **5.3.1 Data in Column Names**

In the **`billboard`** dataset:

- Each **row** represents a **song**
- **artist**, **track**, and **date.entered** describe the song
- **wk1–wk76** store weekly chart ranks

This structure is **not tidy** because:
- The **column names** represent a variable (**week**)
- The **cell values** represent another variable (**rank**)

To fix this, we pivot the weekly columns into two new variables:  
**week** and **rank**.

Missing values occur because many songs chart for fewer than 76 weeks.  
These **structural NAs** should be removed using **`values_drop_na = TRUE`**.

To make future analysis easier, the **week** variable is converted from text
(e.g., `"wk1"`) to numeric using **`parse_number()`**.

Once tidy, the data can be easily visualized to show how song rankings change
over time.

---

## 🧠 **5.3.2 How Pivoting Works**

Pivoting reshapes data in three key ways:

- **Existing variables** are repeated as needed
- **Column names** become values in a new variable
- **Cell values** are unwound row-by-row

For example, when pivoting blood pressure measurements:
- Patient IDs are repeated
- Measurement names (`bp1`, `bp2`) become values
- Blood pressure readings become a single value column

---

## 🧩 **5.3.3 Multiple Variables in Column Names**

Some datasets encode **multiple variables** inside column names.

In the **`who2`** dataset, column names include:
- **Diagnosis method** (`sp`, `ep`, `rel`)
- **Gender** (`m`, `f`)
- **Age range** (`014`, `1524`, etc.)

Each column name is split into multiple new variables using:
- **`names_to`** with multiple entries
- **`names_sep`** to define how names are split

This produces one column per variable while preserving all values.

---

## 🧬 **5.3.4 Variable Names *and* Values in Headers**

The most complex case occurs when column names contain:
- **Variable names** (e.g., `dob`, `name`)
- **Variable values** (e.g., `child1`, `child2`)

Using the special **`.value`** sentinel in **`names_to`** tells
`pivot_longer()` to:
- Use part of the column name as a **new column name**
- Store the remaining part as a **variable value**

This technique cleanly untangles mixed column headers into tidy form.

---

✨ **Key Takeaway**

> **Tidy data** makes transformation, modeling, and visualization easier.  
> Once data is tidy, the entire **tidyverse** works naturally and efficiently.


In [ ]:
library(tidyverse)

billboard_longer <- billboard |>
  pivot_longer(
    cols = starts_with("wk"),
    names_to = "week",
    values_to = "rank",
    values_drop_na = TRUE
  ) |>
  mutate(
    week = parse_number(week)
  )

billboard_longer |>
  ggplot(aes(x = week, y = rank, group = track)) +
  geom_line(alpha = 0.25) +
  scale_y_reverse()

df <- tribble(
  ~id,  ~bp1, ~bp2,
   "A",  100,  120,
   "B",  140,  115,
   "C",  120,  125
)

df |>
  pivot_longer(
    cols = bp1:bp2,
    names_to = "measurement",
    values_to = "value"
  )

who2 |>
  pivot_longer(
    cols = !(country:year),
    names_to = c("diagnosis", "gender", "age"),
    names_sep = "_",
    values_to = "count"
  )

household |>
  pivot_longer(
    cols = !family,
    names_to = c(".value", "child"),
    names_sep = "_",
    values_drop_na = TRUE
  )


# ↔️ **5.4 Widening Data**

So far, we’ve focused on **`pivot_longer()`**, which solves problems where values are stored in column names.  
Now we pivot (yes, *pivot* 😄) to **`pivot_wider()`**, which does the opposite:

- **Increases columns**
- **Reduces rows**
- Useful when **one observation is spread across multiple rows**

This pattern appears less often in the wild, but it’s very common in **government and administrative data**.

---

## 🏥 **Example: Patient Experience Data**

The `cms_patient_experience` dataset records survey results from medical organizations:

- Each **organization** is the core observational unit
- Each organization appears in **multiple rows**
- Each row corresponds to a **different survey measure**

The key columns are:
- `org_pac_id`, `org_nm` → identify the organization
- `measure_cd` → identifies the type of survey question
- `prf_rate` → the numeric performance score

To make the data tidy, we want:
- **One row per organization**
- **One column per survey measure**

---

## 🔧 **Using `pivot_wider()`**

`pivot_wider()` requires two main inputs:

- **`names_from`** → which column supplies new column names
- **`values_from`** → which column supplies the values

However, we also need to specify **`id_cols`**, the columns that uniquely identify each row.  
Without them, R can’t tell how rows should be combined.

Once provided, `pivot_wider()` correctly collapses the data into one row per organization.

---

## 🧠 **How `pivot_wider()` Works**

Conceptually, widening happens in four steps:

1. Identify **unique column names** (`names_from`)
2. Identify **row identifiers** (`id_cols`)
3. Create an empty output grid
4. Fill in values where possible

If a row/column combination doesn’t exist in the input, the output contains **NA**.

---

## ⚠️ **Duplicate Values Problem**

If more than one input row maps to the same output cell:

- `pivot_wider()` creates **list-columns**
- R warns that values are not uniquely identified

To fix this, you must:
- Identify duplicates
- Decide how to summarize them (e.g., mean, max)
- Or repair the underlying data issue

---

✨ **Key Takeaway**

> Use **`pivot_wider()`** when observations are split across rows and should become columns.  
> Always check that each row–column combination maps to **one value only**.


In [ ]:
library(tidyverse)

cms_patient_experience |>
  distinct(measure_cd, measure_title)

cms_patient_experience |>
  pivot_wider(
    id_cols = starts_with("org"),
    names_from = measure_cd,
    values_from = prf_rate
  )

df <- tribble(
  ~id, ~measurement, ~value,
  "A", "bp1", 100,
  "B", "bp1", 140,
  "B", "bp2", 115,
  "A", "bp2", 120,
  "A", "bp3", 105
)

df |>
  pivot_wider(
    names_from = measurement,
    values_from = value
  )

df <- tribble(
  ~id, ~measurement, ~value,
  "A", "bp1", 100,
  "A", "bp1", 102,
  "A", "bp2", 120,
  "B", "bp1", 140,
  "B", "bp2", 115
)

df |>
  pivot_wider(
    names_from = measurement,
    values_from = value
  )

df |>
  group_by(id, measurement) |>
  summarize(n = n(), .groups = "drop") |>
  filter(n > 1)
